In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from datetime import datetime
import rioxarray
import networkx as nx
from shapely import wkt
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from convertbng.util import convert_bng, convert_lonlat
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from scipy.interpolate import griddata
from scipy.stats import genextreme as gev
from geopandas import GeoDataFrame
from shapely.geometry import Point
from copulas.multivariate import GaussianMultivariate
from scipy.interpolate import interp1d
import statsmodels.distributions.empirical_distribution as edf
import seaborn as sns
from scipy.stats import genextreme as gev
from pathos.multiprocessing import ProcessPool, cpu_count
import statsmodels.api as sm
import shapely
from shapely.ops import nearest_points
import multiprocessing as mp
import geopandas as gpd
from shapely.ops import split, snap
from shapely.ops import nearest_points
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely.geometry import LineString, shape
import statsmodels.api as sm
import numpy as np
import pandas as pd
import statsmodels.api as sm
from matplotlib.lines import Line2D
from scipy.interpolate import Rbf
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import itertools
from shapely.geometry import mapping
import matplotlib.colors as clr
from scipy.stats.mstats import gmean
from copulalib.copulalib import Copula
import xarray as xr
import matplotlib.patches as mpatches
import math
from math import radians, cos, sin, asin, sqrt

In [ ]:
# for each figure (Figure 1/2/3)
## analysis modules
# flood depth intersection
# wind copula
# for flood/wind intersection and disruption per system
# uncertainty
# streamflow random regression
# for drought water balance and disruption per system
# for aed per system
# for utility level risk

# Asset database - Figure 2

In [ ]:
# major_cities = dataframe of major cities in Jamaica
# island = geodataframe of Jamaica outline
# parishes = geodataframe of parishes in Jamaica 
# pipelines_NWC = geodataframe of National Water Comission potable pipelines
# potable_facilities_NWC = geodataframe of National Water Comission potable assets 

plants = potable_facilities_NWC[potable_facilities_NWC['asset_type'].isin(['Filter Plant','River Source','Treatment Plant'])]
pumps = potable_facilities_NWC[potable_facilities_NWC['asset_type'].isin(['Spring','Pump Station','Relift Station','Sump','Booster Station'])]
wells = potable_facilities_NWC[potable_facilities_NWC['asset_type'].isin(['Production Well'])]
other = potable_facilities_NWC[potable_facilities_NWC['asset_type'].isin(['Catchment', 'Reservoir', 'Storage Tank'])]
dams = potable_facilities_NWC[potable_facilities_NWC['LOCATION_x'].isin(['Mona Dam', 'Hermitage Dam'])]

fig, ax = plt.subplots(1, 1, figsize=(17,7), linewidth=0.05)    
island.plot(ax=ax, color='whitesmoke', edgecolor='black', linewidth=0.5)
parishes.plot(ax=ax, color='none', edgecolor='black', linewidth=0.5)
pipelines_NWC.plot(ax=ax, color='gold', edgecolor='gold', linewidth=0.4, alpha=0.8)

ax.scatter(plants['lon'], plants['lat'], color='cyan', marker = 's', edgecolor='k', linewidth=0.5)
ax.scatter(pumps['lon'], pumps['lat'], color='navy', marker = '^', edgecolor='k', linewidth=0.5)
ax.scatter(wells['lon'], wells['lat'], color='mediumorchid', marker = 'd', edgecolor='k', linewidth=0.5)
ax.scatter(other['lon'], other['lat'], color='lightgrey', marker = 'o', edgecolor='k', linewidth=0.5, s=5)
ax.scatter(dams['lon'], dams['lat'], color='r', marker = 's', s=20, edgecolor='k', linewidth=0.5)

ax.set_ylim(600000,720000) 
ax.set_xlim(600000,850000)    
ax.set_title('Potable supply systems')
ax.set_xticks([])
ax.set_yticks([])

pipelines = mpatches.Patch(color='gold', label='Pipelines')
plants = mpatches.Patch(color='cyan', label='Water treatment plants')
pumps = mpatches.Patch(color='navy', label='Pumping stations')
wells = mpatches.Patch(color='mediumorchid', label='Wells')
dams = mpatches.Patch(color='red', label='Dams')

ax.scatter(major_cities['X'], major_cities['Y'], s=30, color='k', marker = 'o', edgecolor='k', linewidth=0.5) #major_cities['pop']*10
for i, txt in enumerate(major_cities['name']):
    ax.annotate(txt, (major_cities['X'][i]+500, major_cities['Y'][i]-1000),c='k', fontsize=10, bbox=dict(boxstyle="round", fc="white", ec="None", alpha=0.8)) #, rotation=45

major_cities = mlines.Line2D([], [], color='k', marker='o', linestyle='None',
                          markersize=10, label='Major cities')
pipelines = mlines.Line2D([], [], color='gold', marker='None', linestyle='-',
                          markersize=10, label='Pipelines')
plants = mlines.Line2D([], [], color='cyan', marker='s', linestyle='None',
                          markersize=10, label='Water treatment plants')
pumps = mlines.Line2D([], [], color='navy', marker='^', linestyle='None',
                          markersize=10, label='Pumping stations')
wells = mlines.Line2D([], [], color='mediumorchid', marker='^', linestyle='None',
                          markersize=10, label='Wells')
dams = mlines.Line2D([], [], color='red', marker='s', linestyle='None',
                          markersize=10, label='Dams')
other = mlines.Line2D([], [], color='lightgrey', marker='o', linestyle='None',
                          markersize=5, label='Other')
households = mpatches.Patch(color='grey', label='households')

ax.legend(handles=[pipelines,plants,pumps,wells,other,dams,major_cities], loc='upper right') #households
plt.rcParams['axes.facecolor']='white'
plt.rcParams['savefig.facecolor']='white'

ax.set_facecolor(('paleturquoise'))

## Flood and wind intersection per return period map

In [ ]:
# extreme_wind_return_period_maps = raster of wind speeds per return period per climate change scenario 
# extreme_fluvial_flood_return_period_maps = raster of flood depth per return period per climate change scenario      
# extreme_pluvial_flood_return_period_maps = raster of flood depth per return period per climate change scenario      

# potable_facilities_NWC = geodataframe of National Water Comission potable assets 

for hazard in [extreme_wind_rp_maps, extreme_fluvial_flood_rp_maps, extreme_pluvial_flood_rp_maps]:
    for col in list(hazard.columns):      
        potable_facilities_NWC.index = range(len(pts))
        coords = [(x,y) for x, y in zip(potable_facilities_NWC.X, potable_facilities_NWC.Y)]

        potable_facilities_NWC[col] = [x for x in extreme_wind_return_period_maps.sample(coords)]
        potable_facilities_NWC[col] = potable_facilities_NWC.apply(lambda x: x[col][0], axis=1)

## Wind and flood copula

In [ ]:
# max_wind_per_hurr = netcdf of maximum wind speed per hurricane track.id extracted from ERA5 layers
# ObsEventRP = flood return period per hurricane track.id simulated by JBA flood model

ObsEventRP = ObsEventRP.merge(max_wind_per_hurr, on=['track.id','event.id','op.id'])[['event.id','track.id','op.id','rp', 'max_wind', 'start.year']].drop_duplicates()
ObsEventRP = ObsEventRP.sort_values(by=['max_wind'], ascending=False)
num_events = ObsEventRP.shape[0]
ObsEventRP['start.year'] = pd.to_numeric(ObsEventRP['start.year'], errors='coerce')
max_year = ObsEventRP['start.year'].max()
min_year = ObsEventRP['start.year'].min()
num_years = iter_char_2_max -  iter_char_2_min
ObsEventRP['rank_wind'] = np.linspace(1, num_events+1, num_events)
ObsEventRP['rp_wind'] = 1/(ObsEventRP['rank_wind']/num_years)
ObsEventRP = ObsEventRP.sort_values(by=['rp'], ascending=False)
ObsEventRP['rank_flood'] = np.linspace(1, num_events+1, num_events)
ObsEventRP['rp_flood'] = 1/(ObsEventRP['rank_flood']/num_years)
ObsEventRP = ObsEventRP[['rp_flood','rp_wind']]

gaussian = GaussianMultivariate()
gaussian.fit(ObsEventRP)
synthetic_data_df = gaussian.sample(10000)
synthetic_data_df['model'] = ['Gaussian']*synthetic_data_df.shape[0]

## Sample flood and wind return period and associated hazard varialbe per asset

In [ ]:
SimEventRP = # dataframe of JBA synthetic flood events and affected zones under baseline conditions
SimEventRP_rcp26_2080s = # dataframe of synthetic flood events and affected zones under rcp2.6 emissions end of century scenario 
SimEventRP_rcp45_2080s = # dataframe of synthetic flood events and affected zones under rcp4.5 emissions end of century scenario 
SimEventRP_rcp85_2080s = # dataframe of synthetic flood events and affected zones under rcp8.5 emissions end of century scenario 

synthetic_data_df = # sampled wind event conditional flood event 

damage_curves_flood_pump = # flood damage curve for pumping stations 
damage_curves_flood_wtw = # flood damage curve for water treatment plants
damage_curves_flood_well = # flood damage curve for wells
damage_curves_wind = # wind damage curve for all water assets

for cc in [SimEventRP,SimEventRP_rcp26_2080s,SimEventRP_rcp45_2080s,SimEventRP_rcp85_2080s]:
    for i in cc['event.id'].drop_duplicates().to_list(): # pluvial and fluvial
        sub = cc[cc['event.id']==i]
        for zone in sub['zone']:
            zone_gdf = sub[sub['zone']==zone]
            event_zone_flood_rp = zone_gdf['rp']
            synthetic_data_df['rp_flood_diff'] = abs(synthetic_data_df[synthetic_data_df['rp_flood']-event_zone_flood_rp])
            event_zone_wind_rp = synthetic_data_df[synthetic_data_df['rp_flood_diff']==synthetic_data_df['rp_flood_diff'].min()]['rp_wind'].values[0]    
            affected_assets = potable_facilities_NWC.rio.clip(zone_gdf.geometry.apply(mapping),crs=zone_gdf.geometry.crs)
            affected_assets_rp = potable_facilities_NWC.merge(affected_assets, on='node.id')
            for asset in affected_assets_rp['node.id']:
                asset_sub = affected_assets_rp[affected_assets_rp['node.id']==asset]
                # where rp_min and rp_max are the return periods below and above event_zone_flood_rp
                # flood_depth_rp_min and flood_depth_rp_max are the flood depths associated with rp_min and rp_max extracted from the return period maps
                # flood_depth_rp_min and flood_depth_rp_max are the flood depths associated with rp_min and rp_max extracted from the return period maps
                
                asset_sub['flood_depth']  = asset_sub['flood_depth_rp_min'] + (np.log(event_zone_flood_rp) - np/log(rp_min))/(np.log(rp_max) - np.log(rp_min)) * (df['flood_depth_rp_max'] - df['flood_depth_rp_min'])
                # same as above for wind using GEV scaling 
                if asset_sub['asset_type'].isin(['Spring','Pump Station','Relift Station','Sump','Booster Station']):
                    asset_sub['flood_damage_frac'] = damage_curves_flood_pump['dam_frac_rp_min'] + (asset_sub['flood_depth'] - asset_sub['flood_depth_rp_min'])/(asset_sub['flood_depth_rp_max'] - asset_sub['flood_depth_rp_min']) * (damage_curves_flood_pump['dam_frac_rp_max'] - damage_curves_flood_pump['dam_frac_rp_min'])
                elif asset_sub['asset_type'].isin(['Filter Plant','River Source','Treatment Plant']):
                    asset_sub['flood_damage_frac'] = damage_curves_flood_wtw['dam_frac_rp_min'] + (asset_sub['flood_depth'] - asset_sub['flood_depth_rp_min'])/(asset_sub['flood_depth_rp_max'] - asset_sub['flood_depth_rp_min']) * (damage_curves_flood_wtw['dam_frac_rp_max'] - damage_curves_flood_wtw['dam_frac_rp_min'])
                elif asset_sub['asset_type'].isin(['Production Well']):
                    asset_sub['flood_damage_frac'] = damage_curves_flood_well['dam_frac_rp_min'] + (asset_sub['flood_depth'] - asset_sub['flood_depth_rp_min'])/(asset_sub['flood_depth_rp_max'] - asset_sub['flood_depth_rp_min']) * (damage_curves_flood_well['dam_frac_rp_max'] - damage_curves_flood_well['dam_frac_rp_min'])                   
                else:
                    asset_sub['flood_damage_frac'] = 'nan'
                
                asset_sub['wind_damage_frac'] = damage_curves_wind['dam_frac_rp_min'] + (asset_sub['flood_depth'] - asset_sub['flood_depth_rp_min'])/(asset_sub['flood_depth_rp_max'] - asset_sub['flood_depth_rp_min']) * (damage_curves_wind['dam_frac_rp_max'] - damage_curves_wind['dam_frac_rp_min'])
                
                asset_sub['flood_damage_frac_lower'] = asset_sub['flood_damage_frac']-asset_sub['flood_damage_frac']*0.3
                asset_sub['flood_damage_frac_upper'] = asset_sub['flood_damage_frac']+asset_sub['flood_damage_frac']*0.3
                
                asset_sub['duration_of_flood_days'] = asset_sub.apply(lambda row: np.where(row['frac_flood_dam']<0.001, 0,
                                                                    np.where(row['frac_flood_dam']<0.1, 0+(1/30)*((row['frac_flood_dam'])/(0.1)),
                                                                    np.where(row['frac_flood_dam']<0.2, 1/30+(3/30-1/30)*((row['frac_flood_dam']-0.1)/(0.2-0.1)),
                                                                    np.where(row['frac_flood_dam']<0.3, 3/30+(7/30-3/30)*((row['frac_flood_dam']-0.2)/(0.3-0.2)),
                                                                    np.where(row['frac_flood_dam']<0.35, 7/30+(0.25-7/30)*((row['frac_flood_dam']-0.3)/(0.35-0.3)),
                                                                    np.where(row['frac_flood_dam']>=0.35, 0.25+(1-0.25)*((row['frac_flood_dam']-0.35)/(0.4-0.35)),1)))))), axis=1)
                asset_sub['duration_of_event_flood_upper'] = asset_sub.apply(lambda row: np.where(row['frac_flood_dam_upper']<0.001, 0,
                                                                    np.where(row['frac_flood_dam_upper']<0.1, 0+(1/30)*((row['frac_flood_dam_upper'])/(0.1)),
                                                                    np.where(row['frac_flood_dam_upper']<0.2, 1/30+(3/30-1/30)*((row['frac_flood_dam_upper']-0.1)/(0.2-0.1)),
                                                                    np.where(row['frac_flood_dam_upper']<0.3, 3/30+(7/30-3/30)*((row['frac_flood_dam_upper']-0.2)/(0.3-0.2)),
                                                                    np.where(row['frac_flood_dam_upper']<0.35, 7/30+(0.25-7/30)*((row['frac_flood_dam_upper']-0.3)/(0.35-0.3)),
                                                                    np.where(row['frac_flood_dam_upper']>=0.35, 0.25+(1-0.25)*((row['frac_flood_dam_upper']-0.35)/(0.4-0.35)),1)))))), axis=1)
                asset_sub['duration_of_event_flood_lower'] = asset_sub.apply(lambda row: np.where(row['frac_flood_dam_lower']<0.001, 0,
                                                                    np.where(row['frac_flood_dam_lower']<0.1, 0+(1/30)*((row['frac_flood_dam_lower'])/(0.1)),
                                                                    np.where(row['frac_flood_dam_lower']<0.2, 1/30+(3/30-1/30)*((row['frac_flood_dam_lower']-0.1)/(0.2-0.1)),
                                                                    np.where(row['frac_flood_dam_lower']<0.3, 3/30+(7/30-3/30)*((row['frac_flood_dam_lower']-0.2)/(0.3-0.2)),
                                                                    np.where(row['frac_flood_dam_lower']<0.35, 7/30+(0.25-7/30)*((row['frac_flood_dam_lower']-0.3)/(0.35-0.3)),
                                                                    np.where(row['frac_flood_dam_lower']>=0.35, 0.25+(1-0.25)*((row['frac_flood_dam_lower']-0.35)/(0.4-0.35)),1)))))), axis=1)
                
                asset_sub['CDD_flood_asset'] = asset_sub['duration_of_event_days']*num_days_in_month*asset_sub['population_served_by_asset']
                asset_sub['CDD_flood_asset_min'] = asset_sub['duration_of_event_days_upper']*num_days_in_month*asset_sub['population_served_by_asset']
                asset_sub['CDD_flood_asset_max'] = asset_sub['duration_of_event_days_lower']*num_days_in_month*asset_sub['population_served_by_asset']
                system = asset_sub.groupby('system')['CDD_asset'].max().to_frame().reset_index().rename(columns = {'CDD_asset':'CDD_system'})
                system['CDD_flood_system_max'] = asset_sub.groupby('system')['CDD_flood_asset_min'].max().to_frame().reset_index()['CDD_flood_asset_min']
                system['CDD_flood_system_min'] = asset_sub.groupby('system')['CDD_flood_asset_max'].min().to_frame().reset_index()['CDD_flood_asset_max']
                # same as above for wind 
                
                system['CDD_system'] = system[['CDD_flood_asset','CDD_wind_asset']].max(axis=1)
                system['CDD_system_max'] = system[['CDD_flood_system_max','CDD_wind_asset_max']].max(axis=1)
                system['CDD_system_min'] = system[['CDD_flood_system_min','CDD_wind_asset_min']].max(axis=1)

# Random regression

In [ ]:
data_RF = # read dataframe of era5 temp and precip, and streamflow per abstraction point:
    
y_rf = data_RF['Monthly Streamflow'] 
x_rf = data_RF[['Temperature','Rainfall','Rainfall_lag1','Rainfall_lag2']]
x_rf_full = monthly_water_supply_ba[['Temperature','Rainfall','Rainfall_lag1','Rainfall_lag2']]
x_rf  = sm.add_constant(x_rf)
x_rf_full  = sm.add_constant(x_rf_full)
X_train = x_rf.tail(384)
y_train = y_rf.tail(384)
X_test = x_rf.head(96)
y_test = y_rf.head(96)
        
model_RF = RandomForestRegressor(n_estimators=1000, bootstrap = True,max_features = 'sqrt')
model_RF.fit(X_train, y_train)

rf_test = model_RF.predict(X_test)
r2_test = r2_score(y_test, rf_test)
print(r2_test)

rf_train = model_RF.predict(x_rf)

# Water balance

In [ ]:
baseline = # dataframe of synthetically generated baseline monthly supply per systems, population served and leakage loss rate per system  
rcp26 = # dataframe of synthetically generated rcp2.6 monthly supply per systems, population served and leakage loss rate per system  
rcp45 = # dataframe of synthetically generated rcp4.5 monthly supply per systems, population served and leakage loss rate per system  
rcp85 = # dataframe of synthetically generated rcp8.5 monthly supply per systemsm population served and leakage loss rate per system  
for cc in [baseline, rcp26, rcp45, rcp85]:
    df_full_pivot_m = cc_dict[cc]
    df_full_pivot_m_gb = df_full_pivot_m.groupby(['system'])['supply (m3/month)'].quantile(0.01).to_frame().reset_index().rename(columns = {'supply (m3/month)':'env flow (m3/month)'})

    df_full_pivot_m = df_full_pivot_m.merge(df_full_pivot_m_gb, on=['system'])
    df_full_pivot_m['demand (m3/month)'] = (consumption_per_cap_per_month_m3)*df_full_pivot_m['population_served_system']
    df_full_pivot_m['leakage (m3/month)'] = df_full_pivot_m['demand (m3/month)'] * system_leakage_rate
    
    df_full_pivot_m['drought threshold (m3/month)'] = (df_full_pivot_m['demand (m3/month)']+df_full_pivot_m['leakage (m3/month)'])
    df_full_pivot_m['drought threshold (m3/month)_min'] = df_full_pivot_m['drought threshold (m3/month)']-df_full_pivot_m['drought threshold (m3/month)']*0.3
    df_full_pivot_m['drought threshold (m3/month)_max'] = df_full_pivot_m['drought threshold (m3/month)']+df_full_pivot_m['drought threshold (m3/month)']*0.3

    df_full_pivot_m['balance (m3/month)'] = df_full_pivot_m['supply (m3/month)'] + df_full_pivot_m['storage increase (m3/month)'] - df_full_pivot_m['drought threshold (m3/month)']
    df_full_pivot_m['deficit (m3/month)'] = np.where(df_full_pivot_m['balance (m3/month)']<0,df_full_pivot_m['balance (m3/month)']*-1,0)
    df_full_pivot_m['frac'] = (df_full_pivot_m['deficit (m3/month)']/df_full_pivot_m['demand (m3/month)'])
    df_full_pivot_m['frac'] = np.where(df_full_pivot_m['frac']>1, 1, df_full_pivot_m['frac'])

    df_full_pivot_m['balance (m3/month)_min'] = df_full_pivot_m['supply (m3/month)'] + df_full_pivot_m['storage increase (m3/month)'] - df_full_pivot_m['drought threshold (m3/month)_max']
    df_full_pivot_m['deficit (m3/month)_min'] = np.where(df_full_pivot_m['balance (m3/month)_min']<0,df_full_pivot_m['balance (m3/month)_min']*-1,0)
    df_full_pivot_m['frac_min'] = (df_full_pivot_m['deficit (m3/month)_min']/df_full_pivot_m['demand (m3/month)'])
    df_full_pivot_m['frac_min'] = np.where(df_full_pivot_m['frac_min']>1, 1, df_full_pivot_m['frac_min'])

    df_full_pivot_m['balance (m3/month)_max'] = df_full_pivot_m['supply (m3/month)'] + df_full_pivot_m['storage increase (m3/month)'] - df_full_pivot_m['drought threshold (m3/month)_min']
    df_full_pivot_m['deficit (m3/month)_max'] = np.where(df_full_pivot_m['balance (m3/month)_max']<0,df_full_pivot_m['balance (m3/month)_max']*-1,0)
    df_full_pivot_m['frac_max'] = (df_full_pivot_m['deficit (m3/month)_max']/df_full_pivot_m['demand (m3/month)'])
    df_full_pivot_m['frac_max'] = np.where(df_full_pivot_m['frac_max']>1, 1, df_full_pivot_m['frac_max'])

    df_full_pivot_m['CDD'] = df_full_pivot_m['frac']*df_full_pivot_m['population_served_system']*num_days_in_month
    df_full_pivot_m['CDD_min'] = df_full_pivot_m['frac_min']*df_full_pivot_m['population_served_system']*num_days_in_month
    df_full_pivot_m['CDD_max'] = df_full_pivot_m['frac_max']*df_full_pivot_m['population_served_system']*num_days_in_month
    
    df_full_pivot_m_gb = df_full_pivot_m_full.groupby(['system','cc','year','synth'])['CDD'].sum().to_frame().reset_index()
    df_full_pivot_m_gb['CDD_min'] = df_full_pivot_m_full.groupby(['system','cc','year','synth'])['CDD_min'].sum().to_frame().reset_index()['CDD_min']
    df_full_pivot_m_gb['CDD_max'] = df_full_pivot_m_full.groupby(['system','cc','year','synth'])['CDD_max'].sum().to_frame().reset_index()['CDD_max']

## System level aed 

In [ ]:
df_full_pivot_m_full = # dataframe of CDD per asset damage / water shortage event

df_full_pivot_m_gb_aed = df_full_pivot_m_gb.groupby(['system','cc'])['CDD'].apply(list).to_frame().reset_index()
df_full_pivot_m_gb_aed_min = df_full_pivot_m_gb.groupby(['system','cc'])['CDD_min'].apply(list).to_frame().reset_index()
df_full_pivot_m_gb_aed_max = df_full_pivot_m_gb.groupby(['system','cc'])['CDD_max'].apply(list).to_frame().reset_index()
aep_list = []
for i in np.linspace(1,1000,1000):
    aep_list.append(1/(1000/i))

df_full_pivot_m_gb_aed['aep_list'] = [aep_list]*df_full_pivot_m_gb_aed.shape[0]
df_full_pivot_m_gb_aed['aed'] = df_full_pivot_m_gb_aed.apply(lambda row: integrate.trapz(sorted(row['CDD_system'], reverse=True), row['aep_list']), axis=1)

df_full_pivot_m_gb_aed_min['aep_list'] = [aep_list]*df_full_pivot_m_gb_aed_min.shape[0]
df_full_pivot_m_gb_aed_min['aed_min'] = df_full_pivot_m_gb_aed_min.apply(lambda row: integrate.trapz(sorted(row['CDD_system_min'], reverse=True), row['aep_list']), axis=1)

df_full_pivot_m_gb_aed_max['aep_list'] = [aep_list]*df_full_pivot_m_gb_aed_max.shape[0]
df_full_pivot_m_gb_aed_max['aed_max'] = df_full_pivot_m_gb_aed_max.apply(lambda row: integrate.trapz(sorted(row['CDD_system_max'], reverse=True), row['aep_list']), axis=1)

# Figure 6

In [ ]:
system_service_areas = # geodataframe of NWC potable systems

system_service_areas_merge_ad = # dataframe of annual expected asset damage CDD per climate scenario 
system_service_areas_merge_ad = pd.merge(system_service_areas[['system','geometry']], system_service_areas_merge_fld[['system','aed_total_cost','cc']], on='system')
system_service_areas_merge_ad = gpd.GeoDataFrame(system_service_areas_merge_fld,crs="EPSG:3448",geometry=system_service_areas_merge_fld['geometry'])
system_service_areas_merge_ad['aed (thousands)'] = system_service_areas_merge_ad['aed']/1e3

system_service_areas_merge_ws = # dataframe of  annual expected water shortage CDD per climate scenario 
system_service_areas_merge_ws = pd.merge(system_service_areas[['system','geometry']], system_service_areas_merge_win[['system','aed_total_cost','cc']], on='system')
system_service_areas_merge_ws = gpd.GeoDataFrame(system_service_areas_merge_win,crs="EPSG:3448",geometry=system_service_areas_merge_win['geometry'])
system_service_areas_merge_ws['aed (thousands)'] = system_service_areas_merge_ws['aed']/1e3

fig, ax = plt.subplots(1, 2, figsize=(17,3.5), linewidth=0.5) ### 4 grid per return period       
island.plot(ax=ax[0], color='none', edgecolor='black', linewidth=0.5)
system_service_areas_merge_baseline = system_service_areas_merge_ws[system_service_areas_merge_ws['cc']=='baseline'].drop_duplicates()
system_service_areas_merge_baseline.plot(ax=ax[0], column = 'aed (thousands)', cmap='Reds', vmin = min(system_service_areas_merge_baseline['aed (thousands)'].min(),system_service_areas_merge_baseline['aed (thousands)'].min()), 
                                                                                                vmax = max(system_service_areas_merge_baseline['aed (thousands)'].quantile(0.95),system_service_areas_merge_baseline['aed (thousands)'].quantile(0.95)), 
                                                                                                edgecolor='k', alpha=0.5, linewidth=0.5, legend=True, legend_kwds = {'label':'Annual expected CDD (thousands)'}) 
ax[0].set_ylim(600000,720000) 
ax[0].set_xlim(600000,850000)    
ax[0].set_title('Baseline Customer Disruption Days caused by water shortage')
ax[0].set_xticks([])
ax[0].set_yticks([])

island.plot(ax=ax[1], color='none', edgecolor='black', linewidth=0.5)
system_service_areas_merge_baseline = system_service_areas_merge_ad[system_service_areas_merge_ad['cc']=='baseline'].drop_duplicates()
system_service_areas_merge_baseline.plot(ax=ax[1], column = 'aed_total_cost', cmap='Blues', vmin = min(system_service_areas_merge_baseline['aed (thousands)'].min(),system_service_areas_merge_baseline['aed (thousands)'].min()), 
                                                                                                vmax = max(system_service_areas_merge_baseline['aed (thousands)'].quantile(0.95),system_service_areas_merge_baseline['aed (thousands)'].quantile(0.95)), 
                                                                                                edgecolor='k', alpha=0.5, linewidth=0.5, legend=True, legend_kwds = {'label':'Annual expected CDD (thousands)'})
ax[1].set_ylim(600000,720000) 
ax[1].set_xlim(600000,850000)    
ax[1].set_title('Baseline Customer Disruption Days caused by asset damage')
ax[1].set_xticks([])
ax[1].set_yticks([])
plt.tight_layout()

# Utility total return period curve (Figure 3)

In [ ]:
event_df = # dataframe of CDD caused by each asset damage / water shortage event per climate scenario

cc_dict = {'baseline':'lightgrey','rcp26':'lightpink','rcp45':'orange','baseline':'darkred'}
fig, ax = plt.subplots(1, 1, figsize=(15,5), sharey=True)  
for cc in cc_dict:    
    event_df_gb_ba = event_df[event_df['cc']=='baseline']
    
    event_df_gb = event_df_ba.groupby(['cc','event'])['CDD_system'].sum().to_frame().reset_index()
    event_df_gb['CDD_system_max'] = event_df_ba.groupby(['cc','event'])['CDD_system_max'].sum().to_frame().reset_index()['CDD_system_max']
    event_df_gb['CDD_system_min'] = event_df_ba.groupby(['cc','event'])['CDD_system_min'].sum().to_frame().reset_index()['CDD_system_min']
    
    event_df_gb = event_df_gb.sort_values(by=['CDD_system'], ascending=False)
    num_years = 1e3
    num_events = event_df_gb.shape[0]                              
    event_df_gb['rank'] = np.linspace(1, num_events+1, num_events)
    event_df_gb['rp'] = 1/(event_df_gb['rank']/num_years)

    ax.scatter(event_df_gb['rp'],(event_df_gb['CDD_system']/1e6),c=cc_dict[cc], s=25, edgecolor='k', linewidths=0.05, alpha=0.5)    
    ax.fill_between(event_df_gb['rp'],event_df_gb['CDD_system_max']/1e6,event_df_gb['CDD_system_min']/1e6,color=cc_dict[cc], alpha=0.4) 
    
baseline_patch = mpatches.Patch(color='lightgrey', label='baseline')
scen_26_patch = mpatches.Patch(color='orange', label='RCP26')
scen_45_patch = mpatches.Patch(color='lightpink', label='RCP45')
scen_85_patch = mpatches.Patch(color='darkred', label='RCP85')

ax.legend(handles=[baseline_patch,scen_26_patch,scen_45_patch,scen_85_patch])
ax.grid(which='major',linestyle='-', linewidth='0.5', color='silver')
ax.grid(which='minor',linestyle='-', linewidth='0.2', color='silver')
ax.set_xscale('log')
ax.set_xlim(1,1e3)
ax.set_xlabel('Return Period (years)')    
ax.set_ylim(1,event_df_gb['CDD_system_max'].max())
ax.set_ylabel('Customer Disruption Days \n (millions)')
ax2 = ax.twinx()
ax2.set_ylim(0,event_df_gb['CDD_system_max'].max()/(num_utility_customers*num_days_in_year))
ax2.set_ylabel('Customer Disruption Days \n (% of maximum annual disruption of full customer-base)')

# Figure 5 (heatmap)

In [ ]:
event_df = # dataframe of CDD caused by each asset damage / water shortage event per climate scenario

parish_groupby = event_df.groupby(['cc','PARISH','event'])['CDD_system'].sum().to_frame().reset_index()
parish_groupby['parish_population'] = event_df.groupby(['cc','PARISH','event'])['population_served_system'].sum().to_frame().reset_index()['parish_population']
parish_groupby = parish_groupby.merge(asset_criticality_parish, on='PARISH')
parish_groupby['frac_parish_disrupted'] = parish_groupby['CDD_system']/parish_groupby['parish_population']

parish_y = []
parish_x = []
regression = []
for x in parish_groupby['PARISH'].drop_duplicates().to_list():
    iter_x = parish_groupby[parish_groupby['PARISH'] == x]
    iter_x = iter_x[iter_x['frac_parish_disrupted']>0]
    for y in parish_groupby['PARISH'].drop_duplicates().to_list(): 
        iter_y = parish_groupby[parish_groupby['PARISH'] == y]
        iter_y = iter_y[iter_y['parish_frac']>0]
        merged = pd.merge(iter_x, iter_y, on='index')
        parish_x.append(x)
        parish_y.append(y)
        r = np.corrcoef(merged['parish_frac_x'], merged['parish_frac_y'])[1, 0]
        regression.append(r)
        
widespread_df = pd.DataFrame({'parish_x':parish_x, 'parish_y':parish_y, 'regression':regression})
     
widespread_df_pivot = widespread_df.pivot(index='parish_x', columns='parish_y', values='regression')
widespread_df_pivot = widespread_df_pivot.fillna(0)

sns.set(rc={'figure.figsize':(10,8), 'font.sans-serif': ['Arial'], 
            'axes.spines.left': True,
            'axes.spines.bottom': True,
            'axes.spines.right': True,
            'axes.spines.top': True})
sns.set_style("white")

mask = np.zeros_like(widespread_df_pivot,dtype = np.bool)
mask[np.triu_indices_from(mask)] = True

fig, ax = plt.subplots(1, 1, figsize=(8,6), linewidth=0.5) ### 4 grid per return period       
ax = sns.heatmap(widespread_df_pivot, mask = mask, cmap = 'Reds',square = True, vmin = 0, vmax = 1)
plt.xlabel('Parish')
plt.ylabel('Parish')
plt.tight_layout() 

# Figure 4 (bar)

In [ ]:
df = # read dataframe of  asset damage and water shortage-related annual expected CDD per asset type

aran = np.arange(4) 
width = 0.3
fig, ax = plt.subplots(1, 1, figsize=(7,7))

baseline_ad_w = [df[(df['disruption type']=='asset damage')&(df['haz']=='wind')]['baseline'].to_list()[0],
                df[(df['disruption type']=='asset damage')&(df['haz']=='wind')]['rcp26'].to_list()[0],
                df[(df['disruption type']=='asset damage')&(df['haz']=='wind')]['rcp45'].to_list()[0],
                df[(df['disruption type']=='asset damage')&(df['haz']=='wind')]['rcp85'].to_list()[0]]
baseline_ad_f = [df[(df['disruption type']=='asset damage')&(df['haz']=='flood')]['baseline'].to_list()[0],
                df[(df['disruption type']=='asset damage')&(df['haz']=='flood')]['rcp26'].to_list()[0],
                df[(df['disruption type']=='asset damage')&(df['haz']=='flood')]['rcp45'].to_list()[0],
                df[(df['disruption type']=='asset damage')&(df['haz']=='flood')]['rcp85'].to_list()[0]]
rects1 = ax.bar(aran-0.15, baseline_ad_w, width, label='Wind-induced asset damage', color='midnightblue') #yerr = baseline_ad_w_yerr, 
rects1_2 = ax.bar(aran-0.15, baseline_ad_f, width, bottom = baseline_ad_w, label='Flood inundation-induced asset damage', color='lightblue')

baseline_dr = [df[(df['disruption type']=='water shortage')&(df['haz']=='drought')]['baseline'].to_list()[0],
                df[(df['disruption type']=='water shortage')&(df['haz']=='drought')]['rcp26'].to_list()[0],
                df[(df['disruption type']=='water shortage')&(df['haz']=='drought')]['rcp45'].to_list()[0],
                df[(df['disruption type']=='water shortage')&(df['haz']=='drought')]['rcp85'].to_list()[0]]

rects2 = ax.bar(aran+0.15, baseline_dr, width, label='Water shortage', color='darkred') # yerryerr = baseline_dr_yerr, 
ax.set_ylabel('Annual Expected CDD (millions)')
ax.set_title('Customer Disruption Days per climate change scenario (millions)')
ax.set_xticks([0, 1, 2, 3, 4])
ax.set_xticklabels(['Baseline', 'RCP2.6', 'RCP4.5', 'RCP8.5', ''])
ax.set_xlim(right=3.5)

ax.legend()
plt.tight_layout()